In [1]:
# W4D4    polyglot

####     read file  ( change to a function later)

In [2]:
import re
import glob
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfTransformer as TF
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.pipeline import Pipeline

In [3]:
lines = []
webpage_words = []
with open('Python.txt') as f:
    lines.append(f.read())

with open('Python2.txt') as f:
    lines.append(f.read())
    
with open('python_NOT.txt') as f:
    webpage_words.append(f.read())

In [4]:
a = str(lines)
b = re.sub("[^a-zA-Z]\S\W", " ", a)
c = b.split(' ')
# a

In [5]:
# c

In [6]:
# len(c)


In [7]:
num = c.count('')
for i in range(num):
    c.remove('')

####    test & fit it

In [8]:
X_python_test = c
y_python_test = webpage_words

In [9]:
keywords_python = ['del', 'whileas', 'elif', 'global', 'withassert', 'yieldbreak', 'printclass', 'exec', 
                   'in', 'raisecontinue', 'returndef', 'lambda']

In [10]:
mmore = ".count( .index( .append( .extend( .insert( .remove( .pop( .reverse( .sort("
more = mmore.split(" ")

In [11]:
for each in more:
    keywords_python.append(each)

In [12]:
X_python_train = keywords_python
y_python_train = ['python'] * len(X_python_train)

In [13]:
py_pipeline = Pipeline([("count", CV()), ("tfid", TF()), ("multi", MNB())])

In [14]:
py_pipeline.fit(X_python_train, y_python_train)

/Users/RobertWard/Documents/TIY/Week4/polyglot/.direnv/python-3.5.1/lib/python3.5/site-packages/sklearn/naive_bayes.py:420: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_)


Pipeline(steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...ear_tf=False, use_idf=True)), ('multi', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [15]:
py_pipeline.score(X_python_train,y_python_train)
# py_pipeline.score(X_python_test,)

1.0

In [16]:
py_pipeline.predict(X_python_test)   # this is a real python program

array(['python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'python',
       'python', 'python', 'python', 'python', 'python', 'pyth

In [17]:
py_pipeline.predict(y_python_test)   # this is NOT a real python program

array(['python'], 
      dtype='<U6')